# DSCI 511: Data acquisition and pre-processing<br>Chapter 8: Establishing a Database with Documentation

## Exercises
Note: numberings refer to the main notes.

### 8.2.2 Getting the data right, the first time
While accessing a large dataset it might not seem convient to pre-define and execute a database structure, preprocess data, or generate metadata, but it can really save a lot of time on the back end, and should definitely be a prioroty with ongoing (streaming) data collections. Let's revisit out song lyrics exercise and do some strategic preprocessing and file management.

#### 8.2.1.1 Exercise: Building a song lyrics database with metadata
Let's work with the song lyrics scraper we created during the Harvesting Data lecture. Instead of downloading the entire dataset again and then creating our metadata files, it'd be much more efficient to rewrite our data acquisition procedure to create the metadata as it runs&mdash;upon download, we alread have each piece of data interpreted in memory, i.e.,  don't have to read from disk! The old pieces of web scraping code was just storing songs by artist in large, alphabetic data files. Here, our tasks center around making sure the songs are organized alphabeticaly by artist, and are accessible by albums and genres. We'll want to exercise care as we create data and metadata files&mdash;we need to come up with a consistent naming scheme for the different artists and songs since they don't have IDs from the website&mdash;what could go wrong if we just named files according to artist, album, song, or genre names?

Pulling the pieces of scaper code together, here's a fill-in the blanks-style exercise. Complete the marked changes:

In [1]:
from bs4 import BeautifulSoup
import requests, re, string, json, os

#######################################################################
####### 0. Create a primary data directory. ###########################
#######################################################################
#######################################################################

#######################################################################
####### 1. Create reverse-lookup for songs by genre ###################
#######################################################################
#######################################################################

## go through all of the letters in the alphabet
for letter in string.ascii_lowercase:
    
    #######################################################################
    ####### 2. Create the letter-level directory ##########################
    #######################################################################
    #######################################################################
    
    #######################################################################
    ####### 3. Initialize a letter-level metadata file ####################
    ####### create a data file for the current letter
    filename = "songlyrics-{}.json".format(letter)
    fh = open(filename,  "w")
    fh.close()
    #######################################################################
    #######################################################################
    
    ## open and parse the html for the current letter
    letter_link = 'http://www.songlyrics.com/{}/'.format(letter)
    letterhtml = requests.get(letter_link).text
    lettersoup = BeautifulSoup(letterhtml, 'html.parser')

    ## collect the pages for this letter
    pages = ["/{}/".format(letter)]
    for letterlink in lettersoup.find_all('a'):
        ## filter links for letter pages
        if letterlink.get("href") and re.search("^Page \d+$", letterlink.get("title", "NOTITLE")):            
            pages.append(letterlink['href'])

    ## go through the letter pages
    for page in pages:        
        ## open and parse the html for the current page of this letter
        pagehtml = requests.get("http://www.songlyrics.com" + page).text
        pagesoup = BeautifulSoup(pagehtml, 'html.parser')

        ## go through the artists in the page
        for pagelink in pagesoup.find_all('a'):
            ## filter links for artist pages
            if re.search("^http://.*?-lyrics/$", pagelink.get("href", "NOLINK")):

                #######################################################################                
                ####### 4. remove old data structure and hold on to the artist's name 
                ####### set up data and store artist-level information
                data = {
                    "Artist": pagelink.text,
                    "url": pagelink['href'],
                    "Songs": {}
                }
                #######################################################################
                #######################################################################

                #######################################################################
                ####### 5. Output artist info to letter-level metadata file ###########
                #######################################################################
                #######################################################################

                #######################################################################
                ####### 6. Create artist-level directory. #############################
                #######################################################################
                #######################################################################
                
                #######################################################################
                ####### 7. Create an artist-level metadata file #######################
                #######################################################################
                #######################################################################      
                
                ## open and parse the html for the current artist on this page
                artisthtml = requests.get(data["url"]).text
                artistsoup = BeautifulSoup(artisthtml, 'html.parser')                        

                ## go through the songs of this artist
                for songlink in artistsoup.find_all('a'):

                    ## filter links for song pages
                    if songlink.get("itemprop", "NOITEMPROP") == "url" and songlink.get("title"):
                                                
                        #######################################################################
                        ############ 8. Hold song title; store info as artist-level metadata
                        ############ store initial song-level information
                        title = songlink.text
                        data["Songs"][title] = {"Title": title}
                        data["Songs"][title]["url"] = songlink['href']
                        #######################################################################
                        #######################################################################                        

                        ## open and parse the html for the current song by this artist
                        songhtml = requests.get(data["Songs"][title]["url"]).text
                        songsoup = BeautifulSoup(songhtml, 'html.parser')

                        ## go through paragraphs to find song attributes
                        for par in songsoup.find_all("p"):
                            if re.search(": ", par.text):
                                pieces = re.split(": ", par.text)
                                key = pieces[0]
                                value = ": ".join(pieces[1: len(pieces)])

                                #######################################################################                                
                                ############ 9. add song attributes to artist-level metadata ##########
                                data["Songs"][title][key] = value    
                                #######################################################################
                                #######################################################################                        

                                #######################################################################                                
                                ############ 10. add song attributes to reverse song lookup ###########
                                #######################################################################
                                #######################################################################                                

                        #######################################################################                                
                        ############ 11. output song metadata to artist-level metadata file ###
                        #######################################################################
                        #######################################################################                                
                                
                        ## go through divs to find the one with the song lyrics
                        for div in songsoup.find('body').find_all('div'):
                            if div.get("id","NOCLASS") == "songLyricsDiv-outer":
                                
                                #######################################################################                                
                                ############ 12. output song lyrics as text in artist-level directory #                                
                                data["Songs"][title]["Lyrics"]=div.text
                                #######################################################################
                                #######################################################################
                        
                        break
                        
                #######################################################################
                #### 13. remove old data write out ####################################
                
                ## write out the data for this artist, appending to the end of this letter's file
                with open(filename, "a") as fh:
                    fh.writelines(json.dumps(data)+"\n")
                    
                #######################################################################
                #######################################################################
                
                break
        break
        
    break

#######################################################################
####### 14. Output reverse-lookup for songs by attributes #############
#######################################################################
#######################################################################

#### 8.2.1.2 Solution: Making the changes (Spoilers if you're doing the exercise!)
By making a reverse lookup metadata file for artists by associated song attributes, we are denormalizing data and making it easy to perform specific transformations which are interesting for analysis. So here is the above scraper, with the additional changes we wanted to make all filled out. This is a lot, so make sure to take some time and really figure out what's going on here. If you run this code, take the time as well to review the directory structure and files it creates, and if you performed the above exercise on your own, compare your edits to those below!

In [9]:
from bs4 import BeautifulSoup
import requests, re, string, json, os

#######################################################################
####### 0. Create a primary data directory. ###########################
os.system("mkdir ./data/")
#######################################################################
#######################################################################

#######################################################################
####### 1. Create objects for reverse-lookup of songs by genre ########
songsByAttribute = {}
attributeIDs = {}
attributes = {}
attributeNumbers = {}
#######################################################################
#######################################################################

## go through all of the letters in the alphabet
for letter in string.ascii_lowercase:

    numartists = 0
    
    #######################################################################
    ####### 2. Create the letter-level directory ##########################
    os.system("mkdir ./data/{}/".format(letter))
    #######################################################################
    #######################################################################    
    
    #######################################################################
    ####### 3. Initialize a letter-level metadata file ####################
    lettermetafile = "./data/{}/lettermeta.json".format(letter)
    fh = open(lettermetafile,  "w")
    fh.close()    
    #######################################################################
    #######################################################################
    
    ## open and parse the html for the current letter
    letter_link = 'http://www.songlyrics.com/{}/'.format(letter)
    letterhtml = requests.get(letter_link).text
    lettersoup = BeautifulSoup(letterhtml, 'html.parser')

    ## collect the pages for this letter
    pages = ["/{}/".format(letter)]
    for letterlink in lettersoup.find_all('a'):
        ## filter links for letter pages
        if letterlink.get("href") and re.search("^Page \d+$", letterlink.get("title", "NOTITLE")):            
            pages.append(letterlink['href'])

    ## go through the letter pages
    for page in pages:        
        ## open and parse the html for the current page of this letter
        pagehtml = requests.get("http://www.songlyrics.com" + page).text
        pagesoup = BeautifulSoup(pagehtml, 'html.parser')

        ## go through the artists in the page
        for pagelink in pagesoup.find_all('a'):
            ## filter links for artist pages
            if re.search("^http://.*?-lyrics/$", pagelink.get("href", "NOLINK")):
                
                #######################################################################                
                ####### 4. remove old data structure and hold on to the artist's data #
                ####### keep track of number of artists, songs, and create an ID
                numartists += 1
                artistID = "{}-{}".format(letter, str(numartists))
                numsongs = 0
                
                artist = pagelink.text
                artisturl = pagelink['href']
                
                
#                 ## set up data and store artist-level information
#                 data = {
#                     "Artist": pagelink.text,
#                     "url": pagelink['href'],
#                     "Songs": {}
#                 }                
                #######################################################################
                #######################################################################

                #######################################################################                
                ####### 5. Output artist info to letter-level metadata file ###########
                with open(lettermetafile,  "a") as f:
                    f.writelines(artistID + "\t" + artist + "\t" + artisturl + "\n")                    
                #######################################################################
                #######################################################################                    
                    
                #######################################################################
                ####### 6. Create artist-level directory. #############################
                artist_dir = './data/{}/{}/'.format(letter, artistID)
                os.system("mkdir " + artist_dir)
                #######################################################################
                #######################################################################
                
                #######################################################################
                ####### 7. Create an artist-level metadata file #######################
                artistmetafile = artist_dir + "artistmeta.json"
                fh = open(artistmetafile,  "w")
                fh.close()               
                #######################################################################
                #######################################################################                
                                
                ## open and parse the html for the current artist on this page
                ## note we now use the artist's url!
                artisthtml = requests.get(artisturl).text
                artistsoup = BeautifulSoup(artisthtml, 'html.parser')                        

                ## go through the songs of this artist
                for songlink in artistsoup.find_all('a'):

                    ## filter links for song pages
                    if songlink.get("itemprop", "NOITEMPROP") == "url" and songlink.get("title"):                        

                        #######################################################################
                        ############ 8. Hold song title; store info as artist-level metadata ##
                        ## keep track of number of songs and create and ID
                        numsongs += 1
                        titleID = "{}-{}".format(artistID, str(numsongs))
                        
                        ## hold on to the song's title
                        title = songlink.text
                        
#                         data["Songs"][title] = {"Title": title}
#                         data["Songs"][title]["url"] = songlink['href']

                        data = {
                            "ID": titleID,
                            "title": title,
                            "url": songlink['href']
                        }
                        #######################################################################
                        #######################################################################

                        ## open and parse the html for the current song by this artist
                        ## note the data format has changed to get the song's url!
                        songhtml = requests.get(data["url"]).text
                        songsoup = BeautifulSoup(songhtml, 'html.parser')

                        ## go through paragraphs and get song attributes
                        for par in songsoup.find_all("p"):
                            if re.search(": ", par.text):
                                pieces = re.split(": ", par.text)
                                key = pieces[0]
                                value = ": ".join(pieces[1: len(pieces)])

                                #######################################################################                                
                                ############ 9. add song attributes to artist-level metadata ##########
                                if key != "Note":
                                    data[key] = value
                                #######################################################################
                                #######################################################################
                                
                                #######################################################################                                
                                ############ 10. add song attributes to reverse song lookup ###########
                                if key != "Note":
                                    attributeNumbers.setdefault(key, 1)
                                    attributeIDs.setdefault(key, {})
                                    attributes.setdefault(key, {})
                                    if not attributeIDs[key].get(value, False):
                                        attributeID = "{}-{}".format(key, str(attributeNumbers[key]))
                                        attributes[key][attributeID] = value
                                        attributeIDs[key][value] = attributeID
                                        attributeNumbers[key] += 1
                                    else:
                                        attributeID = attributeIDs[key][value]                                        
                                    
                                    songsByAttribute.setdefault(key, {})
                                    songsByAttribute[key].setdefault(attributeID, {})
                                    songsByAttribute[key][attributeID].setdefault(artistID, [])
                                    songsByAttribute[key][attributeID][artistID].append(titleID)
                                #######################################################################
                                #######################################################################

                        #######################################################################                                
                        ############ 11. output song metadata to artist-level metadata file ###
                        with open(artistmetafile,  "a") as f:
                            f.writelines(json.dumps(data) + "\n")
                        #######################################################################
                        #######################################################################                            
                                
                        ## go through divs to find the one with the song lyrics
                        for div in songsoup.find('body').find_all('div'):
                            if div.get("id", "NOCLASS") == "songLyricsDiv-outer":

                                #######################################################################                                
                                ############ 12. output song lyrics as text in artist-level directory #
                                title_file = "./data/{}/{}/{}.txt".format(letter, artistID, titleID)
                                with open(title_file, "w") as f:
                                    f.writelines(div.text + "\n")
                                
#                                 data["Songs"][title]["Lyrics"]=div.text
                                #######################################################################
                                #######################################################################

                                break
            
                    ## now, only break after 10 songs by an artist
                    if numsongs >= 1:
                        break
                        
                #######################################################################
                #### 13. remove old data write out ####################################
#                 ## write out the data for this artist, appending to the end of this letter's file
#                 with open(filename, "a") as fh:
#                     fh.writelines(json.dumps(data)+"\n")
                #######################################################################
                #######################################################################
                
            ## now, only break if this is the tenth artist of this letter!
            if numartists >= 1:
                break
        
        ## this stops us after one page of each letter
        break
        
    ## this stops us after one letter in the alphabet
#     break

#######################################################################
####### 14. Output reverse-lookup for songs by genre ##################
os.system("mkdir ./data/Genre/")
fh = open("./data/Genre/attributeIDs.txt", "w")
for attributeID in songsByAttribute["Genre"]:
    fh.writelines(attributeID + "\t" + attributes["Genre"][attributeID] + "\n")
    with open("./data/Genre/" + attributeID + ".json", "w") as f:
        f.writelines(json.dumps(songsByAttribute["Genre"][attributeID]) + "\n")
fh.close()
#######################################################################
#######################################################################

#### 8.2.2.1 Exercise: accessing songs by album
Review the `genreSongs()` function and use it as a starting point to retrieve a specific albumn's worth of song data for a specified artist. Albums and artists should be specified by string arguments. Be sure to have this functionfail gracefully/informatively if no match is found in the database! 

In [ ]:
## place code here